In [2]:
import pandas as pd 
import numpy as np
from keras.layers import SimpleRNN, Dense, Activation, TimeDistributed
from keras.models import Sequential
ns = pd.read_csv("/work/Indian_Names.csv") 
ns=ns.iloc[:,1]
df = pd.DataFrame({'Name':ns})


df['Name']=df['Name'].astype(str)





2021-08-06 10:27:39.936942: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-06 10:27:39.936993: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:

print(df)

           Name
0         aabid
1        aabida
2        aachal
3        aadesh
4         aadil
...         ...
6496     pratha
6497    preetha
6498    pratham
6499  samriddhi
6500     akshit

[6501 rows x 1 columns]


In [4]:
df['Name'] = df['Name'].apply(lambda x : '\t' + x)
df['target'] = df['Name'].apply(lambda x : x[1:len(x)] + '\n')
print(df)

             Name       target
0         \taabid      aabid\n
1        \taabida     aabida\n
2        \taachal     aachal\n
3        \taadesh     aadesh\n
4         \taadil      aadil\n
...           ...          ...
6496     \tpratha     pratha\n
6497    \tpreetha    preetha\n
6498    \tpratham    pratham\n
6499  \tsamriddhi  samriddhi\n
6500     \takshit     akshit\n

[6501 rows x 2 columns]


In [5]:
def get_vocabulary(names):
    vocabulary = set(['\t', '\n'])
    for name in names:
        for c in name:
            if c not in vocabulary:
                vocabulary.add(c)
                
    return vocabulary

def get_max_len(names):    
    length_list=[]
    for l in names:        
        length_list.append(len(l))    
    max_len = np.max(length_list)
    return max_len



In [6]:
vocabulary = get_vocabulary(df['Name'])
ctoi = { char : idx for idx, char in enumerate(sorted(vocabulary))}
itoc = { idx : char for idx, char in enumerate(sorted(vocabulary))}

max_len = get_max_len(df.Name)

input_data = np.zeros((len(df.Name), max_len+1, len(vocabulary)),dtype='float32')
target_data = np.zeros((len(df.Name), max_len+1, len(vocabulary)),dtype='float32')

In [7]:
for n_idx, name in enumerate(df['Name']):
  for c_idx, char in enumerate(name):
    input_data[n_idx, c_idx, ctoi[char]] = 1.

# Iterate for each name in the dataset
for n_idx, name in enumerate(df['target']):
  for c_idx, char in enumerate(name):
    target_data[n_idx, c_idx, ctoi[char]] = 1.

In [8]:
model = Sequential()
model.add(SimpleRNN(50, input_shape=(max_len+1, len(vocabulary)),return_sequences=True))
model.add(TimeDistributed(Dense(len(vocabulary), activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 21, 50)            4350      
_________________________________________________________________
time_distributed (TimeDistri (None, 21, 36)            1836      
Total params: 6,186
Trainable params: 6,186
Non-trainable params: 0
_________________________________________________________________
2021-08-06 10:28:03.421150: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-06 10:28:03.421440: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-06 10:28:03.421466: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-06 10:28:03.421499: I tenso

In [16]:
model.fit(input_data, target_data, batch_size=128, epochs=32)

Epoch 1/32
51/51 [==============================] - 1s 13ms/step - loss: 0.6831
Epoch 2/32
51/51 [==============================] - 1s 13ms/step - loss: 0.6806
Epoch 3/32
51/51 [==============================] - 1s 12ms/step - loss: 0.6803
Epoch 4/32
51/51 [==============================] - 1s 12ms/step - loss: 0.6802
Epoch 5/32
51/51 [==============================] - 1s 13ms/step - loss: 0.6799
Epoch 6/32
51/51 [==============================] - 1s 12ms/step - loss: 0.6797
Epoch 7/32
51/51 [==============================] - 1s 11ms/step - loss: 0.6797
Epoch 8/32
51/51 [==============================] - 1s 12ms/step - loss: 0.6795
Epoch 9/32
51/51 [==============================] - 1s 12ms/step - loss: 0.6792
Epoch 10/32
51/51 [==============================] - 1s 12ms/step - loss: 0.6795
Epoch 11/32
51/51 [==============================] - 1s 13ms/step - loss: 0.6792
Epoch 12/32
51/51 [==============================] - 1s 12ms/step - loss: 0.6791
Epoch 13/32
51/51 [==================

In [17]:
output_seq = np.zeros((1, max_len+1, len(vocabulary)))
output_seq[0, 0, ctoi['\t']] = 1

# Get the probabilities for the first character
probs = model.predict_proba(output_seq, verbose=0)[:,1,:]


# Sample vocabulary to get first character
first_char = np.random.choice(sorted(list(vocabulary)), replace=False,p=probs.reshape(len(vocabulary)))

# Print the character generated
print(first_char)

# Update the vector to contain first the character
output_seq[0, 1, ctoi[first_char]] = 1.

# Get the probabilities for the second character
probs = model.predict_proba(output_seq, verbose=0)[:,2,:]

# Sample vocabulary to get second character
second_char = np.random.choice(sorted(list(vocabulary)), replace=False,p=probs.reshape(len(vocabulary)))


# Print the second character
print(second_char)

o
n
/shared-libs/python3.7/py/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [18]:
def generate_names(n):
    for i in range(0,n):        
        stop=False        
        counter=1        
        name = ''     
        output_seq = np.zeros((1, max_len+1, len(vocabulary)))        
        output_seq[0, 0, ctoi['\t']] = 1.
        while stop == False and counter < 10:            
            probs = model.predict_proba(output_seq, verbose=0)[:,counter-1,:]       
            c = np.random.choice(sorted(list(vocabulary)), replace=False, p=probs.reshape(len(vocabulary)))
            if c=='\n':                
                stop=True 
            else:                
                name = name + c                
                output_seq[0,counter , ctoi[c]] = 1.                
                counter=counter+1        
        print(name)


In [24]:
generate_names(10)

ramin
raivakhvi
mohav
nikin
bhadi
sarhoo
wejina
savender
charjup
chhadai


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bbd6a6d2-aec1-4ac4-a4d3-96894f078b54' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>